In [1]:
# This program obtains data from the AIF and stores them in the datastore for the required fields

In [2]:
import azureml.core
import pandas as pd
from azureml.core import Workspace, Dataset, Environment
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.47.0 to work with eunmldevamlwsgom


In [3]:
# Loading the Workspace


from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')


eunmldevamlwsgom
seq00963-nprd-eun-mldev-aml-gom
northeurope
cf11c61d-e6ca-4f6b-b8df-d2a77e8a4d04


In [4]:
# Loading the data store

from azureml.core import Datastore
datastore = Datastore.get(workspace=ws, datastore_name="aif_gom")
datastore


In [5]:
# TO INCLUDE
# Config file for Asset Name
# Config file for start and end date
# Mapping of asset name with Code



In [6]:
# Query Name : Loading only the FC and FP 
# 810 ursa
# 
# Date : From this week till end of the year.
# /****** Script for SelectTopNRows command from SSMS  ******/
stringtorun = "SELECT A.ID, A.WORK_ORDER_SRC_ID, A.FACILITY_ID, A.PLANNER_GROUP_CODE, A.MAINTENANCE_PLANNER_GROUP_NAME, A.ASSET_DOMAIN_CODE, A.WBS_ID, A.WBS_CODE, A.WORK_ORDER_NAME, A.BASIC_START_DATE, A.BASIC_START_DATETIME, A.BASIC_END_DATE, A.SCHEDULED_START_DATE, A.BASIC_END_DATETIME, A.SCHEDULED_FINISH_DATE, A.ORIGINAL_LATEST_ALLOWABLE_FINISH_DATE, A.WORK_ORDER_TYPE_CODE, A.LATEST_ALLOWABLE_FINISH_DATE, A.SCHEDULED_START_DATETIME, A.SCHEDULED_FINISH_DATETIME, A.INTEGRATED_ACTIVITY_PLANNING_IMPACT_TYPE_CODE, A.INTEGRATED_ACTIVITY_PLANNING_IMPACT_TYPE_NAME, A.PLANT_CODE, A.FUNCTIONAL_LOCATION, A.ABC_INDICATOR, A.ITEM_SHORT_TEXT, A.MAIN_WORK_CENTER, A.MAIN_WORK_CENTER_DESCRIPTION, A.CONCAT_SYSTEM_STATUS, A.CONCAT_USER_STATUS, A.PRIORITY_CODE, A.SUPERIOR_ACTIVITY, A.WORK_ORDER_CREATED_DATE, A.LATEST_ALLOWABLE_START_DATE, A.ORIGINAL_LATEST_ALLOWABLE_START_DATE, A.WBS_SRC_ID, A.WORK_ORDER_DESCRIPTION, A.WORK_ORDER_REQUESTER_NAME, A.ESTIMATED_COST, B.PLANNED_WORK, B.PLANNED_WORK_UNIT_OF_MEASURE,B.WORK_ACTIVITY_COUNTER, B.WORK_CENTER, B.WORK_CENTER_DESCRIPTION, B.WORK_ACTIVITY_NUMBER FROM DM_WORK.FACT_WORK_ORDER AS A LEFT JOIN DM_WORK.FACT_WORK_OPERATION AS B ON A.ID = B.WORK_ORDER_ID WHERE SCHEDULED_START_DATE >= '1/1/2023' AND PLANNER_GROUP_CODE = 'OLY' "

In [7]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

query = DataPath(datastore,stringtorun)
tabular = Dataset.Tabular.from_sql_query(query, query_timeout=300)
df = tabular.to_pandas_dataframe()
df.head()

Resolving access token for scope "https://database.windows.net//.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


,ID,WORK_ORDER_SRC_ID,FACILITY_ID,PLANNER_GROUP_CODE,MAINTENANCE_PLANNER_GROUP_NAME,ASSET_DOMAIN_CODE,WBS_ID,WBS_CODE,WORK_ORDER_NAME,BASIC_START_DATE,...,WBS_SRC_ID,WORK_ORDER_DESCRIPTION,WORK_ORDER_REQUESTER_NAME,ESTIMATED_COST,PLANNED_WORK,PLANNED_WORK_UNIT_OF_MEASURE,WORK_ACTIVITY_COUNTER,WORK_CENTER,WORK_CENTER_DESCRIPTION,WORK_ACTIVITY_NUMBER
0,96331642,000060857062,3337554.0,OLY,Mars B - Olympus,AD0107,4382122,00454264,3Y Subsea TopHoled & Exploration Wells S,2023-01-13,...,O.US.MR2.OLY.001,None,None,0.0,1.0,H,00000001,EWIMS,eWIMS Confirmation Workcenter,0010
1,104897342,000061714497,3337554.0,OLY,Mars B - Olympus,AD0107,4382102,00454262,(ISIP) SW Hull Strip pump fail PBE-5320,2023-11-30,...,O.US.MR2.OLY.001,None,None,4000.0,6.0,H,00000001,OLYMECH,Mechanic,0025
2,105078912,000061732658,3337554.0,OLY,Mars B - Olympus,AD0107,4382102,00454262,USW Ball & Needle Vlv Need Replaced SW,2023-02-18,...,O.US.MR2.OLY.001,None,None,400.0,4.0,H,00000001,DP231822,"Pourciau, Darryl",0010
3,1684012222,000061761316,3337554.0,OLY,Mars B - Olympus,AD0107,4382102,00454262,CRN2 TY Boom Hoist Component change out,2023-12-03,...,O.US.MR2.OLY.001,None,None,0.0,22.5,H,00000001,OLYCRMEC,Crane Mechanic,0010
4,1684012242,000061761318,3337554.0,OLY,Mars B - Olympus,AD0107,4382102,00454262,CRN3 TY Boom Hoist Component change out,2023-12-03,...,O.US.MR2.OLY.001,None,None,0.0,60.0,H,00000003,OLYBLF,Olympus Bilfinger Maintenance Crew,0020


In [8]:
# Unloading Point

stringtorun = "SELECT A.ID, A.WORK_ORDER_SRC_ID, C.WORK_ORDER_ID,C.WORK_ORDER_NUMBER, C.CREATION_DATETIME, C.WORK_ORDER_OPERATION_NUMBER,  C.RESERVATION_UNLOADING_POINT FROM DM_WORK.FACT_WORK_ORDER AS A LEFT JOIN DM_SUPPLY_CHAIN.FACT_RESERVATION_REQUIREMENT AS C ON A.WORK_ORDER_SRC_ID = C.WORK_ORDER_NUMBER WHERE SCHEDULED_START_DATE >= '1/1/2023' AND PLANNER_GROUP_CODE = 'OLY'"
query = DataPath(datastore,stringtorun)
tabular = Dataset.Tabular.from_sql_query(query, query_timeout=300)
uldf = tabular.to_pandas_dataframe()



In [9]:
# To convert the unloading point to datetime
from dateutil.parser import parse

def is_date(string, fuzzy=True):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
    
def processETAdate(unloadingdf):
    datelist = []
    for cnt in range(0,len(unloadingdf)):
        strtocheck = str(unloadingdf.loc[unloadingdf.index[cnt],'RESERVATION_UNLOADING_POINT'])
        if is_date(strtocheck) == True :
            datelist.append(parse(strtocheck,fuzzy=True))
    if len(datelist) > 0:
        return max(datelist),len(datelist)
    else:
        return None,None
 
    

In [10]:
currin = pd.DataFrame()
for i in uldf['WORK_ORDER_NUMBER'].unique():
    resdf = uldf[uldf['WORK_ORDER_NUMBER']==i]
    for j in resdf['WORK_ORDER_OPERATION_NUMBER'].unique():
        actdf = resdf[resdf['WORK_ORDER_OPERATION_NUMBER']==j]
        if actdf['RESERVATION_UNLOADING_POINT'].str.contains('ETA').any():
            findf = actdf[actdf['RESERVATION_UNLOADING_POINT'].str.contains('ETA')]
            ulp,N = processETAdate(findf)          
            wo = findf.loc[findf.index[-1],'WORK_ORDER_NUMBER']
            ac = findf.loc[findf.index[-1],'WORK_ORDER_OPERATION_NUMBER']
            currin = df[(df['WORK_ORDER_SRC_ID']==wo) & (df['WORK_ACTIVITY_NUMBER']==ac)]
            if len(currin) > 0:
                df.loc[currin.index[0],'ETADate'] = ulp
                df.loc[currin.index[0],'RESERVATION_UNLOADING_POINT_Entries'] = N

In [11]:
# Suitability
stringtorun = 'SELECT SUITABILITY_CODE,WORK_CENTER_LOCATION_NAME, WORK_CENTER_NAME,WORK_CENTER_CODE,PLANT FROM DM_WORK.DIM_WORK_CENTER'
query = DataPath(datastore,stringtorun)
tabular = Dataset.Tabular.from_sql_query(query, query_timeout=300)
suitability_info = tabular.to_pandas_dataframe()


In [12]:
suitability_info.to_csv('suitability_info.csv')

In [13]:
for cnt in df['WORK_CENTER'].unique():
    if cnt != None:
        code = suitability_info[suitability_info['WORK_CENTER_CODE'].str.contains(cnt)]['SUITABILITY_CODE'].unique()
        if len(code)<2:
            subdf = df[df['WORK_CENTER']==cnt]
            df.loc[subdf.index,'SuitabilityCode'] = code[0]

In [14]:
df.to_csv('Olympus_Backlog.csv')

In [15]:
defdatastore = ws.get_default_datastore()
# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df, 
        name='Olympus_Backlog', 
        description='Olympus_Backlog',
        target=defdatastore
    )

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/61c24013-5108-47f3-86ad-f1b4058c9bef/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
